In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
train_df = pd.read_excel("/content/drive/MyDrive/Books/train.xlsx")
eval_df = pd.read_excel("/content/drive/MyDrive/Books/evaluation.xlsx")

In [6]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.0 MB/s eta 0:00:

In [9]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction
from sklearn.metrics import precision_recall_fscore_support, classification_report
from datasets import Dataset
import random

# ... (the rest of the functions remain the same)

# Replace these lines with the actual paths to your train and eval CSV files
# train_df = pd.read_csv("train.csv")
# eval_df = pd.read_csv("eval.csv")
def generate_negatives(df, multiplier=1):
    negative_df = df.copy()
    for _ in range(multiplier):
        negative_df['reason'] = negative_df['reason'].apply(lambda x: ' '.join(random.sample(x.split(), len(x.split()))))
    negative_df['label'] = 0
    return pd.concat([df, negative_df], ignore_index=True)

def preprocess_dataset(df, tokenizer):
    def encode(example):
        inputs = tokenizer(example['text'], example['reason'], padding=True, truncation=True, max_length=512, return_tensors='pt')
        return {k: v.squeeze(0) for k, v in inputs.items()}
    
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(encode, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return dataset



def compute_metrics(eval_pred: EvalPrediction):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {'precision': precision, 'recall': recall, 'f1': f1}

train_df
eval_df

train_df = generate_negatives(train_df, multiplier=1)

models = ['bert-base-uncased', 'distilbert-base-uncased', 'roberta-base']
model_results = {}

for model_name in models:
    print(f"Training and evaluating {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    train_dataset = preprocess_dataset(train_df, tokenizer)
    eval_dataset = preprocess_dataset(eval_df, tokenizer)
    
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        logging_dir=f'./logs/{model_name}',
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()  # This line was missing in your code

    # Error Analysis
    predictions = trainer.predict(eval_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    report = classification_report(eval_dataset['label'], preds, output_dict=True)
    model_results[model_name] = report

print("Error analysis:")
for model_name, report in model_results.items():
    print(f"Model: {model_name}")
    print(report)


Training and evaluating bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Map:   0%|          | 0/8244 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.579700,0.690646,0.000000,0.000000,0.000000
2,0.574500,0.660808,0.000000,0.000000,0.000000
3,0.568100,0.673144,0.000000,0.000000,0.000000


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Error analysis:
Model: bert-base-uncased
{'0': {'precision': 0.5922330097087378, 'recall': 0.010168361393565594, 'f1-score': 0.019993444772205833, 'support': 5999}, '1': {'precision': 0.3325840170844105, 'recall': 0.9860046651116294, 'f1-score': 0.4973945200874096, 'support': 3001}, 'accuracy': 0.33555555555555555, 'macro avg': {'precision': 0.4624085133965742, 'recall': 0.4980865132525975, 'f1-score': 0.25869398242980773, 'support': 9000}, 'weighted avg': {'precision': 0.5056544956125593, 'recall': 0.33555555555555555, 'f1-score': 0.17918018110786432, 'support': 9000}}
Model: distilbert-base-uncased
{'0': {'precision': 0.6326530612244898, 'recall': 0.01033505584264044, 'f1-score': 0.020337871084139737, 'support': 5999}, '1': {'precision': 0.3330712199505729, 'recall': 0.988003998667111, 'f1-score': 0.49819373267243555, 'support': 3001}, 'accuracy': 0.3363333333333333, 'macro avg': {'precision': 0.4828621405875314, 'recall': 0.49916952725487573, 'f1-score': 0.2592658018782876, 'support

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction
from sklearn.metrics import precision_recall_fscore_support, classification_report
from datasets import Dataset
import random

# ... (the rest of the functions remain the same)

def generate_negatives(df, multiplier=1):
    positive_df = df[df['label'] == 1]
    negative_df = df[df['label'] == 0].sample(len(positive_df), replace=True)
    balanced_df = pd.concat([positive_df, negative_df], ignore_index=True)
    return balanced_df.sample(frac=1).reset_index(drop=True)

def preprocess_dataset(df, tokenizer):
    def encode(example):
        inputs = tokenizer(example['text'], example['reason'], padding=True, truncation=True, max_length=512, return_tensors='pt')
        return {k: v.squeeze(0) for k, v in inputs.items()}
    
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(encode, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return dataset



def compute_metrics(eval_pred: EvalPrediction):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {'precision': precision, 'recall': recall, 'f1': f1}

# ... (preprocess_dataset and compute_metrics functions remain the same)

train_df
eval_df

train_df = generate_negatives(train_df, multiplier=1)

models = ['bert-base-uncased', 'distilbert-base-uncased', 'roberta-base']
model_results = {}

for model_name in models:
    print(f"Training and evaluating {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    train_dataset = preprocess_dataset(train_df, tokenizer)
    eval_dataset = preprocess_dataset(eval_df, tokenizer)

    num_pos = len(train_df[train_df['label'] == 1])
    num_neg = len(train_df[train_df['label'] == 0])
    pos_weight = num_neg / num_pos

    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        logging_dir=f'./logs/{model_name}',
        report_to="none",
    )

    model.config.loss_function = 'CrossEntropyLoss'
    model.config.pos_weight = pos_weight

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    # Error Analysis
    predictions = trainer.predict(eval_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    report = classification_report(eval_dataset['label'], preds, output_dict=True)
    model_results[model_name] = report

print("Error analysis:")
for model_name, report in model_results.items():
    print(f"Model: {model_name}")
    print(report)
#This is to address the issue of poor class balance

In [10]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction
from sklearn.metrics import precision_recall_fscore_support, classification_report
from datasets import Dataset
import random

# ... (generate_negatives, preprocess_dataset, and compute_metrics functions remain the same)
def generate_negatives(df, multiplier=1):
    positive_df = df[df['label'] == 1]
    negative_df = df[df['label'] == 0].sample(len(positive_df), replace=True)
    balanced_df = pd.concat([positive_df, negative_df], ignore_index=True)
    return balanced_df.sample(frac=1).reset_index(drop=True)

def preprocess_dataset(df, tokenizer):
    def encode(example):
        inputs = tokenizer(example['text'], example['reason'], padding=True, truncation=True, max_length=512, return_tensors='pt')
        return {k: v.squeeze(0) for k, v in inputs.items()}
    
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(encode, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return dataset



def compute_metrics(eval_pred: EvalPrediction):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {'precision': precision, 'recall': recall, 'f1': f1}



train_df
eval_df

train_df = generate_negatives(train_df, multiplier=1)

models = ['bert-base-uncased', 'distilbert-base-uncased', 'roberta-base', 'sentence-transformers/bert-base-nli-mean-tokens']
model_results = {}

for model_name in models:
    print(f"Training and evaluating {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    train_dataset = preprocess_dataset(train_df, tokenizer)
    eval_dataset = preprocess_dataset(eval_df, tokenizer)

    num_pos = len(train_df[train_df['label'] == 1])
    num_neg = len(train_df[train_df['label'] == 0])
    pos_weight = num_neg / num_pos

    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        num_train_epochs=5,  # Increase the number of epochs
        learning_rate=2e-5,  # Fine-tune the learning rate
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        logging_dir=f'./logs/{model_name}',
        report_to="none",
        lr_scheduler_type='cosine',  # Use cosine learning rate scheduler
    )

    model.config.loss_function = 'CrossEntropyLoss'
    model.config.pos_weight = pos_weight

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    # Error Analysis
    predictions = trainer.predict(eval_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    report = classification_report(eval_dataset['label'], preds, output_dict=True)
    model_results[model_name] = report

print("Error analysis:")
for model_name, report in model_results.items():
    print(f"Model: {model_name}")
    print(report)


Training and evaluating bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Map:   0%|          | 0/4122 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Map:   0%|          | 0/4122 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.257100,3.158566,0.334040,0.997667,0.500502
2,0.127000,3.908104,0.333445,0.996335,0.499666
3,0.079200,3.804852,0.334002,0.998334,0.500543
4,0.075300,4.205793,0.333817,0.996001,0.500042
5,0.059300,4.481509,0.333929,0.997001,0.500293


Error analysis:
Model: bert-base-uncased
{'0': {'precision': 0.6470588235294118, 'recall': 0.005500916819469912, 'f1-score': 0.010909090909090908, 'support': 5999}, '1': {'precision': 0.3333333333333333, 'recall': 0.9940019993335555, 'f1-score': 0.4992468619246862, 'support': 3001}, 'accuracy': 0.33511111111111114, 'macro avg': {'precision': 0.4901960784313726, 'recall': 0.4997514580765127, 'f1-score': 0.25507797641688856, 'support': 9000}, 'weighted avg': {'precision': 0.5424488017429194, 'recall': 0.33511111111111114, 'f1-score': 0.17374260766662442, 'support': 9000}}
Model: distilbert-base-uncased
{'0': {'precision': 0.6666666666666666, 'recall': 0.005334222370395066, 'f1-score': 0.010583760542417728, 'support': 5999}, '1': {'precision': 0.3334450402144772, 'recall': 0.9946684438520493, 'f1-score': 0.4994562034635656, 'support': 3001}, 'accuracy': 0.3352222222222222, 'macro avg': {'precision': 0.5000558534405719, 'recall': 0.5000013331112222, 'f1-score': 0.2550199820029917, 'support

In [11]:
import pandas as pd

# Prepare the data for the table
data = []
for model_name, report in model_results.items():
    precision = report['1']['precision']
    recall = report['1']['recall']
    f1_score = report['1']['f1-score']
    data.append([model_name, precision, recall, f1_score])

# Create the DataFrame and set the column names
results_df = pd.DataFrame(data, columns=['Model', 'Precision', 'Recall', 'F1 Score'])

# Display the DataFrame
print(results_df)


                                             Model  Precision    Recall  \
0                                bert-base-uncased   0.333333  0.994002   
1                          distilbert-base-uncased   0.333445  0.994668   
2                                     roberta-base   0.334002  0.999334   
3  sentence-transformers/bert-base-nli-mean-tokens   0.333929  0.997001   

   F1 Score  
0  0.499247  
1  0.499456  
2  0.500668  
3  0.500293  


To analyze the errors made by each model and discuss possible reasons for these errors, you can start by examining the misclassified examples. This can be done by comparing the ground truth labels with the model's predictions. You can use the following code snippet to get the misclassified examples for each model:

## Error Analysis

In [12]:
misclassified_examples = {}

for model_name in models:
    predictions = trainer.predict(eval_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    misclassified_indices = np.where(eval_dataset['label'] != preds)[0]
    misclassified_examples[model_name] = eval_df.iloc[misclassified_indices]


In [13]:
for model_name, misclassified_df in misclassified_examples.items():
    print(f"\nMisclassified examples for {model_name}:")
    print(misclassified_df)



Misclassified examples for bert-base-uncased:
                                      text                       reason  label
0  the app is crashing when i play a vedio  app crashes during playback      1

Misclassified examples for distilbert-base-uncased:
                                      text                       reason  label
0  the app is crashing when i play a vedio  app crashes during playback      1

Misclassified examples for roberta-base:
                                      text                       reason  label
0  the app is crashing when i play a vedio  app crashes during playback      1

Misclassified examples for sentence-transformers/bert-base-nli-mean-tokens:
                                      text                       reason  label
0  the app is crashing when i play a vedio  app crashes during playback      1
